## SASPy Tabulation for Descriptive Statistics

This notebook demonstrates the usage of a powerful set of tools for descriptive statistics and nesting data in SASPy, powered by the TABULATE procedure.

In [1]:
import saspy
sas = saspy.SASsession(cfgname='default')

SAS Connection established. Subprocess id is 12892



In [2]:
cars = sas.sasdata('cars', 'sashelp')
cars.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945,33337,3.5,6,265,17,23,4451,106,189
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820,21761,2.0,4,200,24,31,2778,101,172
2,Acura,TSX 4dr,Sedan,Asia,Front,26990,24647,2.4,4,200,22,29,3230,105,183
3,Acura,TL 4dr,Sedan,Asia,Front,33195,30299,3.2,6,270,20,28,3575,108,186
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755,39014,3.5,6,225,18,24,3880,115,197


## Basic usage

Like the TABULATE procedure on which it relies, using the **tabulate** methods attached to your SASPy data sets means specifying three things: 

  1. **class columns**, by which to group your data; 
  2. **var columns**, which contain data to be calculated; and 
  3. **statistics**, to be calculated on the var columns within groupings of data.
   
Then you compose the table using a simple syntax, in which `*` indicates a nesting and `|` indicates elements at the same level (this is made possible by Python's operator overloading).

In [3]:
# define columns to use as classes or computational vars
by_origin, by_type     = cars.tabulate.classes('origin', 'type')
horsepower, cylinders  = cars.tabulate.vars('horsepower', 'cylinders')

# grab statistics of interest
mean, n                = cars.tabulate.stats('mean', 'n')

# compose these elements into a table
cars.tabulate.table(
    left = by_origin * by_type,
    top  = (horsepower | cylinders) * (mean | n)
)

In [4]:
# alternatively, you can output pure-text tables using .text_table()
cars.tabulate.text_table(
    left = by_origin * by_type,
    top  = (horsepower | cylinders) * (mean | n)
)


                                                           The SAS System                           14:46 Friday, August 3, 2018   1

                       „ƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒ…ƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒ…ƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒ†
                       ‚                               ‚       Horsepower        ‚        Cylinders        ‚
                       ‚                               ‡ƒƒƒƒƒƒƒƒƒƒƒƒ…ƒƒƒƒƒƒƒƒƒƒƒƒˆƒƒƒƒƒƒƒƒƒƒƒƒ…ƒƒƒƒƒƒƒƒƒƒƒƒ‰
                       ‚                               ‚    Mean    ‚     N      ‚    Mean    ‚     N      ‚
                       ‡ƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒ…ƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒˆƒƒƒƒƒƒƒƒƒƒƒƒˆƒƒƒƒƒƒƒƒƒƒƒƒˆƒƒƒƒƒƒƒƒƒƒƒƒˆƒƒƒƒƒƒƒƒƒƒƒƒ‰
                       ‚Origin         ‚Type           ‚            ‚            ‚            ‚            ‚
                       ‡ƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒˆƒƒƒƒƒƒƒƒƒƒƒƒƒƒƒ‰            ‚            ‚            ‚            ‚
                       ‚Asia           ‚Hybrid         ‚       92.00‚        3.00‚        3.67‚       

### Adding options

Labels, formats (for aggregate statistics), and total groupings ('all' keyword) may also be specified, either when creating a placeholder or during composition using .with_()

In [5]:
# assign labels, formats
by_origin, by_type     = cars.tabulate.classes('origin', 'type', labels=False)
horsepower, cylinders  = cars.tabulate.vars('horsepower', 'cylinders')
mean, n                = cars.tabulate.stats('mean', 'n', formats=['6.2', '3.'], labels=['Average', 'Count'])

# you can override or add options in composition by using .with_()
cars.tabulate.table(
    left = by_origin.with_(all='Total') * by_type,
    top  = (horsepower.with_(label='Power!') | cylinders) * (mean.with_(format='6.2') | n)
)

### Alternative: Create class, var, and stat elements individually

In [6]:
by_origin   = cars.tabulate.as_class('origin', label='', all='All Origins')
by_type     = cars.tabulate.as_class('type', label='')

horsepower  = cars.tabulate.as_var('horsepower')
average     = cars.tabulate.stat('mean', format='8.2', label='Mean HP')
n           = cars.tabulate.stat('n', label='Count', format='6.')


cars.tabulate.table(
    left = by_origin,
    top  = by_type * (n | horsepower.with_(label='') * average)
)

## Composition and Re-use

The real advantage of a Python interface to PROC TABULATE comes with the ability to compose fragments of interest and then recompose those into various tables at will. 

In [7]:
# create some elements for reuse
by_origin   = cars.tabulate.as_class('origin', label='')
by_type     = cars.tabulate.as_class('type', label='')

horsepower  = cars.tabulate.as_var('horsepower', label='Horses')
cylinders   = cars.tabulate.as_var('cylinders', label='Cyls.')
enginesize  = cars.tabulate.as_var('enginesize', label='Engine Size')

average     = cars.tabulate.stat('mean', label='Avg', format='8.2')
stdev       = cars.tabulate.stat('std', label='Std. Dev.', format='5.2')
n           = cars.tabulate.stat('n', label='Count', format='6.0')

# create some compositional fragments
by_origin_and_type = by_origin.with_(all='All') * by_type.with_(all='All')
hpstats            = horsepower * (average | stdev | n)
cylstats           = cylinders * (average | stdev | n)
enginestats        = enginesize * (average | stdev)

In [8]:
# draw a table
cars.tabulate.table(
    left = by_origin_and_type,
    top  = hpstats
)

In [9]:
# draw another table
cars.tabulate.table(
    left = by_type,
    top  = cylstats | enginestats
)

In [10]:
# grab another class when needed, draw another table
drivetrain = cars.tabulate.as_class('drivetrain', label='Drive Train')

cars.tabulate.table(
    left = by_type * drivetrain.with_(all='All'),
    top  = cylstats | enginestats
)

### Retrieve an indexed DataFrame

One of SASPy's best features is the integration with Pandas DataFrames. Instead of drawing a presentational table in HTML or plain text, you can have the resulting nested values converted to a DataFrame using nested indices. 

Note that certain presentational elements (labels, formats, etc) aren't represented, nor is the exact visual arrangement of your groupings. However, all computations are in the DataFrame and can be further accessed or sliced easily in Python.

In [11]:
# since w
my_frame = cars.tabulate.to_dataframe(
    left = by_type * drivetrain * by_origin,
    top  = cylstats | enginestats
)

# showing an excerpt
my_frame[:10]

Cylinders_Mean  Cylinders_Std  Cylinders_N  \
Type   Origin DriveTrain                                               
Hybrid Asia   Front             3.666667       0.577350            3   
SUV    Asia   All               6.000000       1.460593           16   
              Front             6.000000       1.000000            9   
       Europe All               7.200000       1.032796           10   
       USA    All               6.833333       1.800673           12   
              Front             6.923077       1.037749           13   
Sedan  Asia   All               4.857143       1.069045            7   
              Front             4.846154       0.994490           78   
              Rear              6.888889       1.054093            9   
       Europe All               6.277778       1.017815           18   

                          EngineSize_Mean  EngineSize_Std  
Type   Origin DriveTrain                                   
Hybrid Asia   Front              1.633333        0.321455  
SUV    Asia   All                3.493750        0.944788  
              Front              3.433333        0.951315  
       Europe All                3.950000        0.943104  
       USA    All                4.500000        1.471548  
              Front              4.223077        0.790732  
Sedan  Asia   All                2.785714        0.393398  
              Front              2.510256        0.718510  
              Rear               3.733333        0.665207  
       Europe All                3.144444        0.746933

In [12]:
# you can access portions of that nested frame by indices, from left to right
my_frame.loc[('Sedan','Asia')]

,Cylinders_Mean,Cylinders_Std,Cylinders_N,EngineSize_Mean,EngineSize_Std
DriveTrain,,,,,
All,4.857143,1.069045,7,2.785714,0.393398
Front,4.846154,0.994490,78,2.510256,0.718510
Rear,6.888889,1.054093,9,3.733333,0.665207


In [13]:
my_frame.loc[('SUV')]

Cylinders_Mean  Cylinders_Std  Cylinders_N  \
Origin DriveTrain                                               
Asia   All               6.000000       1.460593           16   
       Front             6.000000       1.000000            9   
Europe All               7.200000       1.032796           10   
USA    All               6.833333       1.800673           12   
       Front             6.923077       1.037749           13   

                   EngineSize_Mean  EngineSize_Std  
Origin DriveTrain                                   
Asia   All                3.493750        0.944788  
       Front              3.433333        0.951315  
Europe All                3.950000        0.943104  
USA    All                4.500000        1.471548  
       Front              4.223077        0.790732

### View the generated code

This can also serve as an excellent tool for teaching the complex syntax of PROC TABULATE statements. 

In [14]:
sas.teach_me_SAS(True)

cars.tabulate.table(
    left = by_type * drivetrain.with_(all='All'),
    top  = cylstats | enginestats
)

proc tabulate data=sashelp.cars ;
  class type drivetrain;
  var enginesize cylinders;
  table type='' * (drivetrain='Drive Train' ALL='All'), (cylinders='Cyls.' * (mean='Avg'*f=8.2 std='Std. Dev.'*f=5.2 n='Count'*f=6.0) enginesize='Engine Size' * (mean='Avg'*f=8.2 std='Std. Dev.'*f=5.2));
run;
